In [36]:
import sqlite3
from tweepy import API
from tweepy import OAuthHandler
from tweepy import Cursor
from tweepy import RateLimitError

import time
import twitter_credentials
import datetime

In [6]:
auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)                          
api = API(auth)

In [63]:
class DatabaseConnection():
    def __init__(self, database_name):
        self.conn = sqlite3.connect(database_name)
        
    def write_conservative(self, info):
        """
        info: 
        """
        sql = ''' INSERT INTO Users(ScreenName, UserId, PolLabel)
              VALUES(?,?,?) '''
        cur = self.conn.cursor()
        cur.execute(sql, info)
        self.conn.commit()
        
    def close(self):
        self.conn.close()
        

In [64]:
conservative_db = 'conservatives.db'
db_connect = DatabaseConnection(conservative_db)


In [65]:
def limit_handled(cursor):
    """                                                                                             
    Handles Twitter API's RateLimitError and StopIteration for Cursor                               
    :param cursor: Tweepy Cursor object                                                             
    :return: None                                                                                   
    """
    while True:
        try:
             yield cursor.next()
        except RateLimitError:
            d1 = datetime.datetime.now() + datetime.timedelta(minutes=15)
            print(d1)
            time.sleep(15 * 60)
        except StopIteration:
            break

In [68]:
def write_followers(username):
    count = 0
    for follower in limit_handled(Cursor(api.followers, screen_name=username).items(6)):
        if follower.protected == False:
            info = (follower.screen_name.lower(), follower.id_str, 'conservative')
            db_connect.write_conservative(info)
            print(count, info[0:2])
            count += 1
            
        


In [69]:
write_followers('GOP')
db_connect.close()

2019-09-02 15:00:59.065698
0 ('kenreed50609142', '1130822957000003584')
1 ('johnnyr57223086', '947908081538797568')
2 ('ryan_racic', '1168651257986408448')
3 ('jay_daugherty1', '1163610471410405376')
4 ('a_d_i_t_h_y_a', '914834067031441408')


In [62]:
db_connect.close()

In [1]:
import DatabaseConnection

In [2]:
db = DatabaseConnection.DatabaseConnection('main.db')

In [3]:
db.write_new_user(('knownastron', '82277769', None, None))

In [4]:
db.write_tweet(('6969696', 'yuh', 'this is a tweet #awesome', '2010-05-01 10:28:13'))